In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import albumentations as A

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

import argparse
import logging
from pathlib import Path

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument("--vide_length", type=int, default=50)
parser.add_argument("--img_size", type=int, default=128)
parser.add_argument("--num_class", type=int, default=13)
parser.add_argument("--epochs", type=int, default=40)
parser.add_argument("--seed", type=int, default=7777)
parser.add_argument("--batch_size", type=int, default=8)
parser.add_argument("--lr", type=str, default=3e-4)

parser.add_argument('--gpu', type=str, default='1')
parser.add_argument("--infer_dir", type=str, default='save_log/0224cutout/best_model.pt')
parser.add_argument("--submit_name", type=str, default='save_log/0223seed/submit0225.csv')

args = parser.parse_args(args=[])

label_dict = {
    -1:[-1,-1,-1,-1],
    0:[0,0,0,0],
    1:[1,1,0,0],
    2:[2,1,0,1],
    3:[3,1,1,0],
    4:[4,1,1,1],
    5:[5,1,2,0],
    6:[6,1,2,1],
    7:[7,2,0,0],
    8:[8,2,0,1],
    9:[9,2,1,0],
    10:[10,2,1,1],
    11:[11,2,2,0],
    12:[12,2,2,1]
}

label_reverse_dict = {
    (0,0,0):0,
    (1,0,0):1,
    (1,0,1):2,
    (1,1,0):3,
    (1,1,1):4,
    (1,2,0):5,
    (1,2,1):6,
    (2,0,0):7,
    (2,0,1):8,
    (2,1,0):9,
    (2,1,1):10,
    (2,2,0):11,
    (2,2,1):12,
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(args.seed) # Seed 고정

os.environ["CUDA_VISIBLE_DEVICES"]= args.gpu  # Set the GPU 1 to use
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
class CustomDataset(Dataset):
    def __init__(self, video_path_list, label_dict, transforms, args, train_mode = 'train'):
        self.video_path_list = video_path_list
        self.label_list = label_dict
        self.mode = train_mode
        self.transforms = transforms
        self.args = args
        
    def __getitem__(self, index):        
        images = self.get_frames(self.video_path_list[index])
                        
        if self.transforms is not None:
            res = self.transforms(**images)
            images = np.zeros((len(images), self.args.img_size, self.args.img_size, 3))
            images[0, :, :, :] = res["image"]
            for i in range(1, len(images)):
                images[i, :, :, :] = res[f"image{i}"]

        images = torch.FloatTensor(images).permute(3, 0, 1, 2)
        if self.label_list is not None:
            label = self.label_list[index]
            return images, label
        else:
            return images

    def __len__(self):
        return len(self.video_path_list) 
    
    def get_frames(self, path):
        cap = cv2.VideoCapture(path)
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        imgs = []        
        for fidx in range(frames):
            _, img = cap.read()            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgs.append(img)
        
        ret = {f"image{i}":imgs[i] for i in range(1, len(imgs))}
        ret['image'] = imgs[0]

        return ret

train_transforms = A.Compose([
    A.Resize(height=args.img_size, width=args.img_size),
    A.HorizontalFlip(p=0.5),
    A.Normalize()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

test_transforms = A.Compose([
    A.Resize(height=args.img_size, width=args.img_size),
    A.Normalize()
], p=1, additional_targets={f"image{i}":"image" for i in range(1, 50)})

In [5]:
class BaseModel(nn.Module):
    def __init__(self, hidden_size=400):
        super(BaseModel, self).__init__()
        self.ego = torchvision.models.video.r3d_18(pretrained=True)   
        self.weather = torchvision.models.video.r3d_18(pretrained=True)
        self.timing = torchvision.models.video.r3d_18(pretrained=True)
        
        self.ego_involve_fc = self.create_linear_layer(hidden_size, 3)
        self.weather_fc = self.create_linear_layer(hidden_size, 3)
        self.timing_fc = self.create_linear_layer(hidden_size, 1)
        
        self.dropout = nn.Dropout(0.5)
        
    def create_linear_layer(self, in_features, out_features, bias=True):
        layer = nn.Linear(in_features, out_features, bias = bias)
        layer.weight.data.normal_(mean=0.0, std=1.0)
        if bias:
            layer.bias.data.zero_()
        return layer
        
    def forward(self, x):
        ego_x = self.dropout(F.relu(self.ego(x)))
        weather_x = self.dropout(F.relu(self.weather(x)))
        timing_x = self.dropout(F.relu(self.timing(x)))
        
        ego_output = self.ego_involve_fc(ego_x)
        weather_output = self.weather_fc(weather_x)
        timing_output = self.timing_fc(timing_x)

        timing_output = torch.sigmoid(timing_output).squeeze()
        
        return ego_output, weather_output, timing_output

In [6]:
test = pd.read_csv('/data/jhjang_datasets/car_crash/test.csv')
test['video_path'] = test['video_path'].apply(lambda x : '/data/jhjang_datasets/car_crash' + x[1:])

test_dataset = CustomDataset(test['video_path'].values, None, test_transforms, args)
test_loader = DataLoader(test_dataset, batch_size = args.batch_size, shuffle=False, num_workers=0)

In [7]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    ego_preds = []
    weather_preds = []
    timing_preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            ego_output, weather_output, timing_output = model(videos)

            ego_preds += ego_output.argmax(1).detach().cpu().numpy().tolist()
            weather_preds += weather_output.argmax(1).detach().cpu().numpy().tolist()
            timing_preds += np.where(timing_output.detach().cpu().numpy() > 0.5, 1, 0).tolist()
            
    return ego_preds, weather_preds, timing_preds

In [8]:
_infer_model = BaseModel()
infer_model = nn.DataParallel(_infer_model).to(device)

infer_model.load_state_dict(torch.load(args.infer_dir))

<All keys matched successfully>

In [9]:
ego_preds, weather_preds, timing_preds = inference(infer_model, test_loader, device)

100%|██████████| 225/225 [08:06<00:00,  2.16s/it]


In [10]:
submit = pd.read_csv('/data/jhjang_datasets/car_crash/sample_submission.csv')

label_split = []
for e,w,t in zip(ego_preds, weather_preds, timing_preds):
    if e == 0:
        label_split.append((0,0,0))
    else:
        label_split.append((e,w,t))

submit['label_split'] = label_split

submit['label'] = submit['label_split'].apply(label_reverse_dict.get)

In [11]:
submit.iloc[:,:2].to_csv(args.submit_name, index=False)

In [12]:
c = pd.read_csv('save_log/0221multi_0.62/submit0221_r18.csv')
c

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1795,TEST_1795,7
1796,TEST_1796,0
1797,TEST_1797,1
1798,TEST_1798,7


In [14]:
c[c['label'] != submit['label']]

,sample_id,label
14,TEST_0014,1
15,TEST_0015,6
128,TEST_0128,6
144,TEST_0144,1
182,TEST_0182,3
...,...,...
1621,TEST_1621,3
1641,TEST_1641,1
1678,TEST_1678,11
1716,TEST_1716,3


In [14]:
c[c['label'] != submit['label']]

,sample_id,label
14,TEST_0014,1
15,TEST_0015,6
128,TEST_0128,6
144,TEST_0144,1
182,TEST_0182,3
...,...,...
1621,TEST_1621,3
1641,TEST_1641,1
1678,TEST_1678,11
1716,TEST_1716,3


In [13]:
submit

,sample_id,label,label_split
0,TEST_0000,0,"(0, 0, 0)"
1,TEST_0001,0,"(0, 0, 0)"
2,TEST_0002,0,"(0, 0, 0)"
3,TEST_0003,0,"(0, 0, 0)"
4,TEST_0004,0,"(0, 0, 0)"
...,...,...,...
1795,TEST_1795,7,"(2, 0, 0)"
1796,TEST_1796,0,"(0, 0, 0)"
1797,TEST_1797,1,"(1, 0, 0)"
1798,TEST_1798,7,"(2, 0, 0)"
